In [12]:
from __future__ import unicode_literals, print_function
import spacy
import plac
from spacy.util import minibatch, compounding
import random
import pickle
from pathlib import Path

In [2]:
training_data = []
file = open('training_data.jsonl', 'r', encoding='utf-8')
for line in file.readlines():
    line = line.rstrip('\n ",')
    line = eval(line) 
    training_data.append(line)

In [41]:
#!/usr/bin/env python
# coding: utf8

new_label = 'POLITICAL PARTY'


output_dir=('/home/miriam/Projects/Arkivverket/AI-Oppgave/')
n_iter=(100)

new_model_name = 'nb_core_news_sm_added_new_ner'
model = spacy.load('nb_core_news_sm')  # load existing spacy model
print("Loaded model '%s'" % model)
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')

ner.add_label(new_label)   # Add new entity labels to entity recognizer
optimizer = nlp.entity.create_optimizer()
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(n_iter):
        random.shuffle(training_data)
        losses = {}
        batches = minibatch(training_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.25, losses=losses)
            print('Losses', losses)

    # Test the trained model
    test_text = 'Senterpartiet får en oppslutning på 22,1 prosent, opp 2,3 prosentpoeng fra november. '
    test_text2 = 'De politiske partiene som er representert på Stortinget i perioden 2017-2021, er Arbeiderpartiet, Fremskrittspartiet, Høyre, Senterpartiet, Kristelig Folkeparti, Sosialistisk Venstreparti, Venstre, Miljøpartiet De Grønne og Rødt.'

    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # Save model 
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # Test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text2)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

Loaded model '<spacy.lang.nb.Norwegian object at 0x7f158096bcd0>'
Losses {'ner': 24.472079396247864}
Losses {'ner': 55.39086908753961}
Losses {'ner': 103.5340828364715}
Losses {'ner': 149.43045134004205}
Losses {'ner': 176.65409040730447}
Losses {'ner': 215.9205257995054}
Losses {'ner': 268.76973361801356}
Losses {'ner': 284.7039377344772}
Losses {'ner': 334.30885882209986}
Losses {'ner': 361.60800785850734}
Losses {'ner': 425.452626417391}
Losses {'ner': 461.1489940388128}
Losses {'ner': 497.2811976177618}
Losses {'ner': 563.5551417460665}
Losses {'ner': 602.6747049978003}
Losses {'ner': 648.1823961408809}
Losses {'ner': 689.7539968164638}
Losses {'ner': 770.1262776525691}
Losses {'ner': 804.367519822903}
Losses {'ner': 858.2939818771556}
Losses {'ner': 900.5229421528056}
Losses {'ner': 932.5550514133647}
Losses {'ner': 988.6659807236865}
Losses {'ner': 1028.0276248725131}
Losses {'ner': 1047.4826885135844}
Losses {'ner': 1083.5144708724692}
Losses {'ner': 1098.4076835485175}
Losses {

Losses {'ner': 10878.513143850047}
Losses {'ner': 10911.688243103701}
Losses {'ner': 10974.572748256403}
Losses {'ner': 11009.939830852229}
Losses {'ner': 11052.062243593413}
Losses {'ner': 47.45294791460037}
Losses {'ner': 115.80877619981766}
Losses {'ner': 149.74816876649857}
Losses {'ner': 195.9820701777935}
Losses {'ner': 223.57105508446693}
Losses {'ner': 265.78262391725}
Losses {'ner': 306.6633171145004}
Losses {'ner': 347.4175388518852}
Losses {'ner': 378.3459379378837}
Losses {'ner': 409.83217272201}
Losses {'ner': 482.6496890489143}
Losses {'ner': 534.4347011748832}
Losses {'ner': 572.3313135806602}
Losses {'ner': 637.5583109561485}
Losses {'ner': 681.7576722730082}
Losses {'ner': 710.3231548654803}
Losses {'ner': 743.0253718520471}
Losses {'ner': 787.3384235221383}
Losses {'ner': 832.1254419404504}
Losses {'ner': 884.4390634614465}
Losses {'ner': 956.6443771440026}
Losses {'ner': 994.1361311990258}
Losses {'ner': 1012.087618501806}
Losses {'ner': 1051.9309269505975}
Losses {'

Losses {'ner': 10869.684075252888}
Losses {'ner': 10906.94596304166}
Losses {'ner': 10949.921999806342}
Losses {'ner': 10987.398133391318}
Losses {'ner': 11024.432500475821}
Losses {'ner': 11106.664199942526}
Losses {'ner': 11133.167397731719}
Losses {'ner': 54.249760150909424}
Losses {'ner': 94.33351564407349}
Losses {'ner': 140.41820734739304}
Losses {'ner': 174.6974818110466}
Losses {'ner': 241.32795041799545}
Losses {'ner': 278.4277256131172}
Losses {'ner': 312.41675144433975}
Losses {'ner': 408.2031436562538}
Losses {'ner': 435.942333817482}
Losses {'ner': 464.3535042107105}
Losses {'ner': 501.47467067837715}
Losses {'ner': 531.2897061407566}
Losses {'ner': 564.7352265417576}
Losses {'ner': 636.2357324659824}
Losses {'ner': 680.8287920057774}
Losses {'ner': 704.707190439105}
Losses {'ner': 765.9268132895231}
Losses {'ner': 825.5143638849258}
Losses {'ner': 875.5082651972771}
Losses {'ner': 900.7455904260278}
Losses {'ner': 930.4476399198174}
Losses {'ner': 967.9279473796487}
Losse

Losses {'ner': 10870.3266626872}
Losses {'ner': 10913.363491205113}
Losses {'ner': 10978.003712372469}
Losses {'ner': 11106.523867325472}
Losses {'ner': 11153.537458138155}
Losses {'ner': 11197.040804104494}
Losses {'ner': 11243.282498614477}
Losses {'ner': 11299.271908061193}
Losses {'ner': 11329.748654620336}
Losses {'ner': 11372.56949575393}
Losses {'ner': 14.230877876281738}
Losses {'ner': 43.43547248840332}
Losses {'ner': 93.41120624542236}
Losses {'ner': 166.38622558116913}
Losses {'ner': 217.3069316148758}
Losses {'ner': 250.23633873462677}
Losses {'ner': 337.51711547374725}
Losses {'ner': 383.6064499616623}
Losses {'ner': 430.71013700962067}
Losses {'ner': 469.09506499767303}
Losses {'ner': 505.8434947133064}
Losses {'ner': 603.04295617342}
Losses {'ner': 643.7964775483997}
Losses {'ner': 667.0642386596592}
Losses {'ner': 719.2266174476536}
Losses {'ner': 760.2764214198978}
Losses {'ner': 799.2419111054332}
Losses {'ner': 872.9308101456554}
Losses {'ner': 908.004807643214}
Loss

Losses {'ner': 10680.658264319536}
Losses {'ner': 10757.880932728884}
Losses {'ner': 10852.91092328274}
Losses {'ner': 10913.883486847517}
Losses {'ner': 10972.41099963629}
Losses {'ner': 11012.612671713468}
Losses {'ner': 11045.934897880194}
Losses {'ner': 11079.894252757666}
Losses {'ner': 11140.581949214575}
Losses {'ner': 11194.03849289858}
Losses {'ner': 11237.323407272932}
Losses {'ner': 11275.536840896246}
Losses {'ner': 11319.826306800482}
Losses {'ner': 11356.831109981176}
Losses {'ner': 11383.443426410791}
Losses {'ner': 32.168198615312576}
Losses {'ner': 81.23710365593433}
Losses {'ner': 140.55000276863575}
Losses {'ner': 156.27331617265008}
Losses {'ner': 207.80552831082605}
Losses {'ner': 244.62910291342996}
Losses {'ner': 269.4950612464454}
Losses {'ner': 295.8717714229133}
Losses {'ner': 327.8111280479934}
Losses {'ner': 379.0856770554092}
Losses {'ner': 400.64794472203357}
Losses {'ner': 441.70982700929744}
Losses {'ner': 473.5046795044909}
Losses {'ner': 507.2841374541

...

Losses {'ner': 6394.679104931798}
Losses {'ner': 6447.386684782949}
Losses {'ner': 6478.894470227329}
Losses {'ner': 6510.303076994984}
Losses {'ner': 6545.034490836231}
Losses {'ner': 6570.290081155865}
Losses {'ner': 6638.52833129224}
Losses {'ner': 6665.031714928715}
Losses {'ner': 6732.782516491978}
Losses {'ner': 6763.319659245579}
Losses {'ner': 6786.607307327358}
Losses {'ner': 6819.8534349327965}
Losses {'ner': 6847.3660186654015}
Losses {'ner': 6877.440754008858}
Losses {'ner': 6903.290613663761}
Losses {'ner': 6978.430001931643}
Losses {'ner': 7018.632606046653}
Losses {'ner': 7067.93001898416}
Losses {'ner': 7098.584980093321}
Losses {'ner': 7125.974008761725}
Losses {'ner': 7173.162640475115}
Losses {'ner': 7220.966923617205}
Losses {'ner': 7274.0699738491385}
Losses {'ner': 7309.202938221773}
Losses {'ner': 7387.657279633364}
Losses {'ner': 7415.460344188294}
Losses {'ner': 7476.394022815308}
Losses {'ner': 7532.438183896622}
Losses {'ner': 7575.486082904419}
Losses {'ner'

Losses {'ner': 5928.568456274224}
Losses {'ner': 5970.038646382047}
Losses {'ner': 6022.144261044217}
Losses {'ner': 6075.225618523313}
Losses {'ner': 6108.355333012296}
Losses {'ner': 6187.364052694989}
Losses {'ner': 6212.846378546907}
Losses {'ner': 6274.795651656343}
Losses {'ner': 6309.657906335546}
Losses {'ner': 6374.091350716306}
Losses {'ner': 6422.432367724134}
Losses {'ner': 6452.463012379361}
Losses {'ner': 6500.105743211461}
Losses {'ner': 6594.4876945435535}
Losses {'ner': 6659.246732991887}
Losses {'ner': 6688.093684953405}
Losses {'ner': 6750.526406091405}
Losses {'ner': 6797.854178313864}
Losses {'ner': 6848.925987367285}
Losses {'ner': 6873.836755120894}
Losses {'ner': 6940.036220395705}
Losses {'ner': 6977.220484876772}
Losses {'ner': 7027.837491178652}
Losses {'ner': 7060.203269386431}
Losses {'ner': 7152.5587499977555}
Losses {'ner': 7175.566457831999}
Losses {'ner': 7241.187702143332}
Losses {'ner': 7332.0770687700715}
Losses {'ner': 7410.141814673087}
Losses {'ne

Losses {'ner': 5951.015481956414}
Losses {'ner': 6034.399418078474}
Losses {'ner': 6106.7553399429835}
Losses {'ner': 6150.869318448118}
Losses {'ner': 6201.840996228269}
Losses {'ner': 6248.993859253935}
Losses {'ner': 6285.495417975}
Losses {'ner': 6312.231215141825}
Losses {'ner': 6363.632664822153}
Losses {'ner': 6404.31487222772}
Losses {'ner': 6465.62879344087}
Losses {'ner': 6540.40212699037}
Losses {'ner': 6576.59185358148}
Losses {'ner': 6608.1733440981425}
Losses {'ner': 6637.782790039777}
Losses {'ner': 6691.946532939672}
Losses {'ner': 6733.210388098954}
Losses {'ner': 6761.101123740076}
Losses {'ner': 6804.212510575651}
Losses {'ner': 6855.901336540579}
Losses {'ner': 6895.966676105856}
Losses {'ner': 6937.035035927891}
Losses {'ner': 6967.890009005665}
Losses {'ner': 6998.33333571732}
Losses {'ner': 7042.821917076706}
Losses {'ner': 7078.423290272354}
Losses {'ner': 7144.492996950745}
Losses {'ner': 7172.937732477783}
Losses {'ner': 7218.50406720221}
Losses {'ner': 7250.8

Losses {'ner': 5916.137450472597}
Losses {'ner': 5976.427255408053}
Losses {'ner': 6004.828217701201}
Losses {'ner': 6061.413801626448}
Losses {'ner': 6105.74235166717}
Losses {'ner': 6142.657753721956}
Losses {'ner': 6167.120552317385}
Losses {'ner': 6220.4924512069265}
Losses {'ner': 6266.883289770369}
Losses {'ner': 6366.610732034926}
Losses {'ner': 6398.280783371214}
Losses {'ner': 6451.926623539213}
Losses {'ner': 6481.481841460947}
Losses {'ner': 6504.722983972315}
Losses {'ner': 6562.451270059828}
Losses {'ner': 6639.216491894011}
Losses {'ner': 6689.886625127081}
Losses {'ner': 6726.8300686638395}
Losses {'ner': 6773.523878888373}
Losses {'ner': 6837.910547332052}
Losses {'ner': 6875.908749838898}
Losses {'ner': 6921.1868096791}
Losses {'ner': 6950.626155038903}
Losses {'ner': 7000.055667956898}
Losses {'ner': 7059.302753766606}
Losses {'ner': 7134.413067539761}
Losses {'ner': 7171.092930515835}
Losses {'ner': 7193.121398647854}
Losses {'ner': 7255.496347626278}
Losses {'ner': 

Losses {'ner': 5940.488468796575}
Losses {'ner': 5992.469365388715}
Losses {'ner': 6024.703985006177}
Losses {'ner': 6052.3795390432715}
Losses {'ner': 6136.956763893926}
Losses {'ner': 6175.4863937800765}
Losses {'ner': 6201.610441477016}
Losses {'ner': 6247.148588688091}
Losses {'ner': 6303.1404943773305}
Losses {'ner': 6356.293365747646}
Losses {'ner': 6443.4790383884465}
Losses {'ner': 6466.837720305637}
Losses {'ner': 6528.596553177551}
Losses {'ner': 6576.8211047360455}
Losses {'ner': 6611.595995874122}
Losses {'ner': 6649.020003051475}
Losses {'ner': 6708.097115428165}
Losses {'ner': 6761.319952399448}
Losses {'ner': 6810.470104129032}
Losses {'ner': 6852.00556150356}
Losses {'ner': 6894.993190440327}
Losses {'ner': 6938.93335464874}
Losses {'ner': 6974.0756926862305}
Losses {'ner': 7016.168896886498}
Losses {'ner': 7070.584955248029}
Losses {'ner': 7110.951354567667}
Losses {'ner': 7146.399690662285}
Losses {'ner': 7180.108906541725}
Losses {'ner': 7239.719358955284}
Losses {'n

In [38]:
from spacy import displacy
text = 'De politiske partiene som er representert på Stortinget i perioden 2017-2021, er Arbeiderpartiet, Fremskrittspartiet, Høyre, Senterpartiet, Kristelig Folkeparti, Sosialistisk Venstreparti, Venstre, Miljøpartiet De Grønne og Rødt.'
nlp2 = spacy.load(output_dir)
doc2 = nlp2(text)
for ent in doc2.ents:
    print(ent.label_, ent.text)
spacy.displacy.render(doc2, style='ent')
